In [29]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Connect to your postgres DB
import psycopg2
from psycopg2 import sql

# Database connection parameters
db_params = {
    "dbname": "FYP",
    "user": "postgres",
    "password": "",       # Apna Apna Password Daalna
    "host": "localhost",  # Change if your database is hosted elsewhere
    "port": "5432"        # Default PostgreSQL port
}

try:
    # Establish a connection to the database
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    
    # Print PostgreSQL version to verify connection
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print("Connected to:", db_version)

except Exception as e:
    print("Error connecting to database:", e)

Connected to: ('PostgreSQL 17.1 on x86_64-windows, compiled by msvc-19.41.34123, 64-bit',)


## Get All Stores Associated with a Distributor
This query fetches all stores associated with a specific distributor, along with their latitude and longitude.

In [31]:
import pandas as pd

In [32]:
def get_stores_by_distributor(connection, distributor_id):
    cursor = connection.cursor()
    query = """
    SELECT 
        ds.distributorid,
        ds.storeid,
        ds.latitude,
        ds.longitude
    FROM 
        distributor_stores ds
    WHERE 
        ds.distributorid = %s;
    """
    cursor.execute(query, (distributor_id,))
    # Fetch all results
    records = cursor.fetchall()
    # Define column names
    columns = ["distributorid", "storeid", "latitude", "longitude"]
    # Convert to DataFrame
    df = pd.DataFrame(records, columns=columns)
    return df

# Example usage
df = get_stores_by_distributor(connection, 654)
df  # Display the DataFrame in Jupyter

,distributorid,storeid,latitude,longitude
0,654,313295,24.8630133,68.3314533
1,654,313337,24.9157544,68.2931230
2,654,313358,24.8627680,68.3312213
3,654,313356,24.9397017,68.2309183
4,654,313347,24.8627547,68.3312263
...,...,...,...,...
135,654,313342,24.8627692,68.3312045
136,654,313352,24.8590522,68.3815374
137,654,313390,24.8629068,68.3313396
138,654,313386,24.8628613,68.3313422


## Fetch All Distributors and Their Total Stores Count
This query fetches all distributors with the count of stores associated with each one, which could be helpful for summary statistics.

In [33]:
def get_distributors_store_count(connection):
    cursor = connection.cursor()
    query = """
    SELECT 
        ds.distributorid,
        COUNT(ds.storeid) AS store_count
    FROM 
        distributor_stores ds
    GROUP BY 
        ds.distributorid;
    """
    cursor.execute(query)
    return cursor.fetchall()
df = pd.DataFrame(get_distributors_store_count(connection), columns=["distributorid", "store_count"])
df

,distributorid,store_count
0,1,395
1,2,336
2,3,513
3,4,221
4,5,412
...,...,...
498,743,418
499,746,443
500,748,44
501,749,517


## Fetch All Active PJPs and Their Associated Distributors
Get all active PJPs along with the distributor information. This can help in understanding the distribution network managed by each PJP.

In [34]:
def get_active_pjps_with_distributors(connection):
    cursor = connection.cursor()
    query = """
    SELECT 
        p.pjpcode,
        p.distributor_ref,
        d.distributorcode
    FROM 
        pjp p
    JOIN 
        distributors d ON p.distributor_ref = d.distributorid;
    """
    cursor.execute(query)
    return cursor.fetchall()
df = pd.DataFrame(get_active_pjps_with_distributors(connection), columns=["pjpcode", "distributor_ref", "distributorcode"])
df

,pjpcode,distributor_ref,distributorcode
0,D0001PJP1,1,D0001
1,D0001PJP10,1,D0001
2,D0001PJP2,1,D0001
3,D0001PJP3,1,D0001
4,D0001PJP4,1,D0001
...,...,...,...
9098,D0745PJP5,750,D0745
9099,D0745PJP6,750,D0745
9100,D0745PJP7,750,D0745
9101,D0745PJP8,750,D0745


## Fetch All Stores Associated with a Specific PJP
Retrieve all stores associated with a particular PJP, including their locations.

In [35]:
def get_stores_by_pjp(connection, pjp_code):
    cursor = connection.cursor()
    query = """
    SELECT 
        us.pjp_ref AS pjpcode,
        us.storecode,
        sl.latitude,
        sl.longitude
    FROM 
        universe_stores us
    JOIN 
        store_hierarchy sh ON us.storecode = sh.storecode
    JOIN 
        store_location sl ON sh.storeid = sl.storeid
    WHERE 
        us.pjp_ref = %s;
    """
    cursor.execute(query, (pjp_code,))
    records = cursor.fetchall()
    # Return a DataFrame with the correct call to pd.DataFrame
    return pd.DataFrame(records, columns=["pjpcode", "storecode", "latitude", "longitude"])

# Example usage
df = get_stores_by_pjp(connection, "D0001PJP1")
df

,pjpcode,storecode,latitude,longitude
0,D0001PJP1,N00000119452,30.0386073,71.8146780
1,D0001PJP1,N00000119453,30.0372403,71.8167720
2,D0001PJP1,N00000095486,30.0372257,71.8230588
3,D0001PJP1,N00000095493,30.0350117,71.8152933
4,D0001PJP1,N00000095515,30.0349550,71.8160633
5,D0001PJP1,N00000000001,30.0382367,71.8142856
6,D0001PJP1,N00000000007,30.0386683,71.8150767
7,D0001PJP1,N00000000013,30.0351235,71.8132982
8,D0001PJP1,N00000000018,30.0384832,71.8138177
9,D0001PJP1,N00000000019,30.0361877,71.8175841


## Get PJP Visit Summary (Total Visits Per PJP)
This query calculates the total number of visits associated with each PJP. This can help you understand the frequency of visits per journey plan.

In [36]:
def get_pjp_visit_summary(connection):
    cursor = connection.cursor()
    query = """
    SELECT 
        p.pjpcode,
        COUNT(v.visitid) AS total_visits
    FROM 
        pjp p
    JOIN 
        visits v ON p.pjpcode = v.pjp_ref
    GROUP BY 
        p.pjpcode;
    """
    cursor.execute(query)
    records = cursor.fetchall()
    columns = ["pjpcode", "total_visits"]
    return pd.DataFrame(records, columns=columns)

# Example usage
df = get_pjp_visit_summary(connection)
df

,pjpcode,total_visits
0,D0001PJP1,350
1,D0001PJP10,262
2,D0001PJP2,322
3,D0001PJP3,352
4,D0001PJP4,132
...,...,...
9077,D0745PJP13,24
9078,D0745PJP19,58
9079,D0745PJP22,32
9080,D0745PJP25,50


## Retrieve All PJPs and Their Order Bookers
Fetch each PJP along with the associated app users (order bookers) responsible for carrying out the visits. This helps in understanding which users are assigned to which journey plans.

In [37]:
def get_pjps_with_order_bookers(connection):
    cursor = connection.cursor()
    query = """
    SELECT 
        p.pjpcode,
        p.appuser_ref AS appuserid,
        a.appusercode
    FROM 
        pjp p
    JOIN 
        app_users a ON p.appuser_ref = a.appuserid;
    """
    cursor.execute(query)
    records = cursor.fetchall()
    columns = ["pjpcode", "appuserid", "appusercode"]
    return pd.DataFrame(records, columns=columns)

# Example usage
df = get_pjps_with_order_bookers(connection)
df

,pjpcode,appuserid,appusercode
0,D0001PJP1,1,D0001OB1
1,D0001PJP10,2,D0001OB2
2,D0001PJP2,1,D0001OB1
3,D0001PJP3,1,D0001OB1
4,D0001PJP4,2,D0001OB2
...,...,...,...
8129,D0745PJP27,8337,D0745OB14
8130,D0745PJP3,8342,D0745OB17
8131,D0745PJP4,8342,D0745OB17
8132,D0745PJP5,8342,D0745OB17


## Calculate Average Visit Time Per PJP
Get the average time spent per visit in each PJP. This can be useful for analyzing efficiency and productivity within each journey plan. (Yeh idk sahi hoga ya nahi coz like fatima mentioned we might not have exact time in and time out)

In [38]:
def get_average_visit_time_per_pjp(connection):
    cursor = connection.cursor()
    query = """
    SELECT 
        v.pjp_ref AS pjpcode,
        AVG(v.visitspenttimeinseconds) AS average_visit_time
    FROM 
        visits v
    GROUP BY 
        v.pjp_ref;
    """
    cursor.execute(query)
    records = cursor.fetchall()
    columns = ["pjpcode", "average_visit_time"]
    return pd.DataFrame(records, columns=columns)

# Example usage
df = get_average_visit_time_per_pjp(connection)
df

,pjpcode,average_visit_time
0,D0001PJP1,30.9114285714285714
1,D0001PJP10,51.5000000000000000
2,D0001PJP2,29.2515527950310559
3,D0001PJP3,33.9090909090909091
4,D0001PJP4,39.6742424242424242
...,...,...
9077,D0745PJP13,162.3333333333333333
9078,D0745PJP19,2.1379310344827586
9079,D0745PJP22,0E-20
9080,D0745PJP25,33.1000000000000000


## Get Store Visit Frequency by PJP
Retrieve the frequency of visits for each store within a specific PJP. This helps to understand how often each store is visited as part of a journey plan.

In [39]:
def get_store_visit_frequency_by_pjp(connection, pjp_code):
    cursor = connection.cursor()
    query = """
    SELECT 
        v.pjp_ref AS pjpcode,
        v.storeid,
        COUNT(v.visitid) AS visit_frequency
    FROM 
        visits v
    WHERE 
        v.pjp_ref = %s
    GROUP BY 
        v.pjp_ref, v.storeid;
    """
    cursor.execute(query, (pjp_code,))
    records = cursor.fetchall()
    columns = ["pjpcode", "storeid", "visit_frequency"]
    return pd.DataFrame(records, columns=columns)

# Example usage
df = get_store_visit_frequency_by_pjp(connection, "D0001PJP1")
df

,pjpcode,storeid,visit_frequency
0,D0001PJP1,1,7
1,D0001PJP1,2,7
2,D0001PJP1,3,7
3,D0001PJP1,4,7
4,D0001PJP1,5,7
5,D0001PJP1,6,7
6,D0001PJP1,7,7
7,D0001PJP1,8,7
8,D0001PJP1,9,7
9,D0001PJP1,10,7
